In [1]:
import uuid

In [2]:
import importlib
from pathlib import Path
from glob import glob

In [3]:
import optuna

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import numpy as np 

import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier

In [5]:
import wandb

In [6]:
from train import prepare_model, train

In [7]:
model = 'sia.models.wickstrom_2020'
dataset = 'sia.datasets.wickstrom_2020'

data_dir = './data/ecg_better_model_physical_activity'

In [8]:
participants = [Path(path).stem for path in glob(f'{data_dir}/*.csv')]
train_participants, test_participants = train_test_split(participants, test_size=0.2, random_state=42)

In [9]:
model_name = model.split('.')[-1]
model_module = importlib.import_module(model)
dataset_module = importlib.import_module(dataset)

In [10]:
sampling_rate = 1000

In [11]:
def objective(trial):
    params = {
        "epochs": 11,
        "num_workers": 8,
        "kfold": 5, 

        "batch_size": 100, #trial.suggest_int("batch_size", 10, 1024),
        "learning_rate": 0.01,

        "window": 30000 # trial.suggest_int("window", 1000, 60000, step=1000),
    }
    params["window_shift"] = 0.25 ##trial.suggest_float("window_shift", 0.1, 1, step=0.05)
    
    wandb.init(
        project='stress-in-action',
        config={
            "epochs": params['epochs'],
            "kfold": params['kfold'],
            "batch_size": params['batch_size'],
            "learning_rate": params['learning_rate'],
            "window": params['window'],
            "window_shift": params['window_shift'],
            "architecture": model_name,
            "dataset": dataset
        }
    )

    scores = []
    for train_indices, val_indices in KFold(n_splits=params['kfold'], shuffle=True, random_state=42).split(train_participants):
        try: 
            model = prepare_model(
                model=model_module.Model, # assuming all models are named Model.
                data=data_dir,
                dataset=dataset_module.Dataset,
                batch_size=params['batch_size'],
                learning_rate=params['learning_rate'],
                num_workers=params['num_workers'],
                ignore_torch_format=False,
                train_participants=[train_participants[i] for i in train_indices],
                val_participants=[train_participants[i] for i in val_indices],
                test_participants=test_participants,
                dataset_kwargs={
                    'window': params['window'],
                    'step_size': int(params['window_shift'] * sampling_rate)
                }
            )

            trainer = train(
                model_name,
                model=model,
                epochs=params['epochs']
            )

            scores.append(trainer.callback_metrics["val_accuracy"].item())
        except Exception as e:
            print(e)
            break

    wandb.finish()

    return np.min([np.mean(scores), np.median([scores])])

study = optuna.create_study(
    study_name=f'{model}_{dataset}_{data_dir}_{str(uuid.uuid4())}',
    storage="sqlite:///db.sqlite3",
    direction='maximize',
    sampler=optuna.samplers.RandomSampler(seed=42)
)

study.optimize(
    objective, 
    n_trials=10,
    show_progress_bar=True
)

[I 2024-05-09 14:35:08,589] A new study created in RDB with name: sia.models.wickstrom_2020_sia.datasets.wickstrom_2020_./data/ecg_better_model_physical_activity_d661b4bd-940f-4938-abda-ae536561a23c
  0%|          | 0/10 [00:00<?, ?it/s]Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alex-antonides. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                 | Type             | Params
-----------------------------------------------------------
0  | l1                   | Sequential       | 1.3 K 
1  | l2                   | Sequential       | 164 K 
2  | l3                   | Sequential       | 98.7 K
3  | gap                  | Sequential       | 0     
4  | out                  | Linear           | 258   
5  | train_accuracy       | BinaryAccuracy   | 0     
6  | train_f1score        | BinaryF1Score    | 0     
7  | train_precision      | BinaryPrecision  | 0     
8  | train_loss           | CrossEntropyLoss | 0     
9  | validation_accuracy  | BinaryAccuracy   | 0     
10 | validation_f1score   | BinaryF1Score    | 0     
11 | validation_precision | BinaryPrecision  | 0     
12 | validation_loss      | CrossEntropyLoss | 0     
13 | test_accuracy        | BinaryAccuracy   | 0     
14 | test_f1score         | BinaryF1Score    | 0     
15 | test_precision       | Binar

Epoch 10: 100%|█████████▉| 12287/12290 [1:30:55<00:01,  2.25it/s, v_num=106]

`Trainer.fit` stopped: `max_epochs=11` reached.


...

 10%|█         | 1/10 [28:44:40<163:43:21, 65489.09s/it]

[W 2024-05-10 19:19:49,068] Trial 1 failed with value nan.
Epoch 3:  24%|██▎       | 2898/12290 [44:27<2:24:05,  1.09it/s, v_num=107]

 20%|██        | 2/10 [28:44:40<109:35:11, 49313.97s/it]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epoch 3:  24%|██▎       | 2898/12290 [44:30<2:24:13,  1.09it/s, v_num=107]


In [ ]:
import wandb

In [ ]:
wandb.finish()